In [1]:
import main.Constants as Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


## Manual Phrase Generator

In [2]:
from main.phrase_generator import *
from main.utils import parse_response_table

In [3]:
gsheet_name = Constants.SHEET_NAME
phrasesheet_name = Constants.PHRASE_SHEET_NAME

phrase_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=phrasesheet_name)
existing_phrases = phrase_dict['Line'].drop_duplicates().values

#### Generate Phrase From Context

In [4]:
translation_model="gpt-4o"
temp=0.7 
num_phrases = 5
complexity = "Medium"

translation_response =  get_completion(
        prompt = get_prompt_to_translate(
            input_phrases = "Do you want to go watch the movie with me?",
            complexity = "Medium",
            tone = "Casual"
            ),
        model=translation_model, 
        temperature=temp)


KeyboardInterrupt: 

In [ ]:
print(translation_response.choices[0].message.content)

| Line                     | Pinyin                          | Meaning                           | Response                | Response Pinyin             | Response Meaning              | Complexity | Category                | Tone   |
|--------------------------|---------------------------------|-----------------------------------|-------------------------|-----------------------------|--------------------------------|------------|-------------------------|--------|
| 你想和我一起去看电影吗？ | Nǐ xiǎng hé wǒ yīqǐ qù kàn diànyǐng ma? | Do you want to go watch the movie with me? | 好啊，几点见？          | Hǎo a, jǐ diǎn jiàn?        | Sure, what time shall we meet? | Medium     | Movies and Entertainment | Casual |


In [ ]:
translation_model="gpt-4o"
temp=0.7 
situation = "Going to the library"
num_phrases = 5
complexity = "Medium"

translation_response =  get_completion(
        prompt = get_prompt_to_gen_phrases(
            situation = "Going to the library",
            num_phrases = 5,
            complexity = "Medium",
            tone = "Casual",
            existing_phrases=existing_phrases
            ),
        model=translation_model, 
        temperature=temp)


In [ ]:
print(translation_response.choices[0].message.content)

| Line            | Pinyin                            | Meaning                         | Response                      | Response Pinyin                    | Response Meaning               | Complexity | Category   |
|------------------|----------------------------------|---------------------------------|-------------------------------|------------------------------------|---------------------------------|------------|------------|
| 这本书在哪个区域？ | Zhè běn shū zài nǎge qūyù?       | Which section is this book in?  | 在三楼的文学区。               | Zài sān lóu de wénxué qū.          | In the literature section on the third floor. | Medium     | Library    |
| 我可以借几本书？   | Wǒ kěyǐ jiè jǐ běn shū?          | How many books can I borrow?    | 你可以借五本书。               | Nǐ kěyǐ jiè wǔ běn shū.             | You can borrow five books.     | Medium     | Library    |
| 图书馆有打印服务吗？| Túshūguǎn yǒu dǎyìn fúwù ma?    | Does the library offer printing services? | 有，在一楼。                 | Yǒu, zài yī lóu.     

In [ ]:
new_phrase_df = parse_response_table(translation_response.choices[0].message.content)
new_phrase_df

,Line,Pinyin,Meaning,Response,Response Pinyin,Response Meaning,Complexity,Category
1,这本书在哪个区域？,Zhè běn shū zài nǎge qūyù?,Which section is this book in?,在三楼的文学区。,Zài sān lóu de wénxué qū.,In the literature section on the third floor.,Medium,Library
2,我可以借几本书？,Wǒ kěyǐ jiè jǐ běn shū?,How many books can I borrow?,你可以借五本书。,Nǐ kěyǐ jiè wǔ běn shū.,You can borrow five books.,Medium,Library
3,图书馆有打印服务吗？,Túshūguǎn yǒu dǎyìn fúwù ma?,Does the library offer printing services?,有，在一楼。,"Yǒu, zài yī lóu.","Yes, on the first floor.",Medium,Library
4,这里的书可以拍照吗？,Zhèlǐ de shū kěyǐ pāizhào ma?,Can I take photos of the books here?,可以，但请不要使用闪光灯。,"Kěyǐ, dàn qǐng bùyào shǐyòng shǎnguāngdēng.","Yes, but please don't use the flash.",Medium,Library
5,有推荐的书吗？,Yǒu tuījiàn de shū ma?,Are there any book recommendations?,当然，这本书很受欢迎。,"Dāngrán, zhè běn shū hěn shòu huānyíng.","Of course, this book is very popular.",Medium,Library


In [ ]:
save_new_phrase_to_dict(new_phrase_df, gsheet_name=gsheet_name, worksheet_name=phrasesheet_name)

'Saved new phrases to the dictionary.'

#### Generate Response from Phrase

In [ ]:
translation_model="gpt-4o"
temp=0.7 

translation_response =  get_completion(
        prompt = get_prompt_to_respond("洗手间在哪里， 今天你要吃什么", complexity="Medium"),
        model=translation_model, 
        temperature=temp)


In [ ]:
new_phrase_df = parse_response_table(translation_response.choices[0].message.content)
new_phrase_df

,Line,Pinyin,Meaning,Response,Response Pinyin,Response Meaning,Complexity,Category
1,洗手间在哪里,Xǐshǒujiān zài nǎlǐ,Where is the restroom?,洗手间在走廊的右边。,Xǐshǒujiān zài zǒuláng de yòubiān.,The restroom is on the right side of the corri...,Medium,Daily Life
2,今天你要吃什么,Jīntiān nǐ yào chī shénme,What do you want to eat today?,我今天想吃中餐。,Wǒ jīntiān xiǎng chī zhōngcān.,I want to eat Chinese food today.,Medium,Cooking


In [ ]:
save_new_phrase_to_dict(new_phrase_df, gsheet_name=gsheet_name, worksheet_name=phrasesheet_name)

'Saved new phrases to the dictionary.'

## Packaging Phrase Generator Pipeline

In [4]:
phrase_generator = PhraseGenerationPipeline(gsheet_name, phrasesheet_name)

In [6]:
translation_model="gpt-4o"
temp=0.7 
translation = "Are you cooking with the oven today?"
complexity = "Medium"
tone = "Casual"

phrase_generator.phrase_translate_module(translation, complexity, tone, translation_model, temp)
phrase_generator.new_phrase_df

,Line,Pinyin,Meaning,Response,Response Pinyin,Response Meaning,Complexity,Category,Tone,Added Date
1,你今天用烤箱做饭吗？,Nǐ jīntiān yòng kǎoxiāng zuò fàn ma?,Are you cooking with the oven today?,是啊，我打算烤个比萨。,"Shì a, wǒ dǎsuàn kǎo gè bǐsà.","Yes, I plan to bake a pizza.",Medium,Cooking,Casual,2025-01-12


: 

In [5]:
translation_model="gpt-4o"
temp=0.7 
situation = "Cooking with my wife"
num_phrases = 2
complexity = "Medium"
tone = "Casual"

phrase_generator.run_phrase_generation_pipeline(situation, num_phrases, complexity, tone, existing_phrases, translation_model, temp)
phrase_generator.new_phrase_df

,Line,Pinyin,Meaning,Response,Response Pinyin,Response Meaning,Complexity,Category,Tone,Added Date,Phrase Id
1,这个菜要怎么做？,Zhège cài yào zěnme zuò?,How do we cook this dish?,我们可以查一下食谱。,Wǒmen kěyǐ chá yīxià shípǔ.,We can check the recipe.,Medium,Cooking,Casual,2025-01-01,34
2,你帮我尝尝这个。,Nǐ bāng wǒ chángcháng zhège.,Help me taste this.,好的，我来试试。,"Hǎo de, wǒ lái shì shì.","Okay, let me try it.",Medium,Cooking,Casual,2025-01-01,35


In [7]:
translation_model="gpt-4o"
temp=0.7 
input_phrases = "明天你要几点出发, 你喜欢看什么电影"
complexity = "Medium"
tone = "Polite"

phrase_generator.phrase_response_module(input_phrases, complexity, tone, translation_model, temp)
phrase_generator.new_phrase_df

,Line,Pinyin,Meaning,Response,Response Pinyin,Response Meaning,Complexity,Category,Tone,Added Date
1,明天你要几点出发,Míngtiān nǐ yào jǐ diǎn chūfā,What time do you plan to leave tomorrow?,我计划早上八点出发。,Wǒ jìhuà zǎoshang bā diǎn chūfā.,I plan to leave at 8 a.m.,Medium,Daily Life,Polite,2025-01-01
2,你喜欢看什么电影,Nǐ xǐhuān kàn shénme diànyǐng,What kind of movies do you like?,我喜欢看科幻电影。,Wǒ xǐhuān kàn kēhuàn diànyǐng.,I like watching science fiction movies.,Medium,Movies and Entertainment,Polite,2025-01-01


In [14]:
translation_model="gpt-4o"
temp=0.7 
situation = "Cooking with my wife"
num_phrases = 2
complexity = "Medium"

phrase_generator.run_phrase_generation_pipeline(situation, num_phrases, complexity, existing_phrases, translation_model, temp)

'Saved new phrases to the dictionary.'

In [15]:
translation_model="gpt-4o"
temp=0.7 
input_phrases = "明天你要几点出发, 你喜欢看什么电影"
complexity = "Medium"

phrase_generator.run_phrase_response_pipeline(input_phrases, complexity, translation_model, temp)

'Saved new phrases to the dictionary.'